# Revisión de código para generación de índices

**Fecha:** 4 de Mayo de 2020

**Responsable de revisión:** León Garay

**Código revisado**

In [2]:
#import cupy as cp
import numpy as np
import pandas as pd
import fix_yahoo_finance as yf
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import time

# Extraer los datos de Yahoo Finance

Tomamos 50 acciones en total, de diferentes industrias:

In [3]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']
len(stocks)

50

In [4]:
df_o = yf.download(stocks, start='2015-01-01', end='2020-04-30').Open
df_o.shape

[*********************100%***********************]  50 of 50 completed


(1664, 50)

In [5]:
df_c = yf.download(stocks, start='2015-01-01', end='2020-04-30').Close
df_c.shape

[*********************100%***********************]  50 of 50 completed


(1664, 50)

In [6]:
df_c.count().describe()

count      50.000000
mean     1338.080000
std        14.255454
min      1270.000000
25%      1340.000000
50%      1340.000000
75%      1340.000000
max      1364.000000
dtype: float64

Como la mayoría tienen 1340 días informados, tomaremos solo eso 1340 días.

In [7]:
base = df_c['AAPL'].dropna().to_frame()

In [8]:
for i in range(0,50):
    base = base.join(df_c.iloc[:,i].to_frame(), lsuffix='_caller', rsuffix='_other')

In [9]:
base = base.drop(columns=['AAPL_caller'])
base = base.rename(columns={"AAPL_other": "AAPL"})
base.head()

,005930.KS,0688.HK,0700.HK,1398.HK,2010.SR,7203.T,AAPL,AI.PA,AMT,AMZN,...,T,UNH,UNP,UPS,V,VOO,VTI,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,26600.0,24.704800,112.800003,5.77,NaN,NaN,109.330002,89.786400,99.669998,308.519989,...,33.869999,100.779999,118.610001,110.379997,66.254997,188.399994,105.919998,46.959999,85.900002,92.830002
2015-01-05,26660.0,24.951799,113.500000,5.80,79.50,7507.0,106.250000,87.005997,98.230003,302.190002,...,33.549999,99.120003,114.599998,108.169998,64.792503,185.089996,104.099998,46.570000,85.650002,90.290001
2015-01-06,25900.0,24.605900,120.000000,5.71,77.00,7300.0,106.260002,86.279999,97.970001,295.290009,...,33.599998,98.919998,112.230003,107.459999,64.375000,183.270004,103.080002,47.040001,86.309998,89.809998
2015-01-07,26140.0,24.507099,124.400002,5.75,78.25,7407.0,107.750000,86.669601,99.000000,298.420013,...,33.169998,99.930000,112.849998,108.459999,65.237503,185.559998,104.309998,46.189999,88.599998,90.720001
2015-01-08,26280.0,23.864799,127.300003,5.72,79.25,7554.0,111.889999,90.317703,99.919998,300.459991,...,33.500000,104.699997,117.080002,110.410004,66.112503,188.820007,106.150002,47.180000,90.470001,92.230003


In [10]:
base.count().describe()

count      50.000000
mean     1324.180000
std        45.854557
min      1051.000000
25%      1340.000000
50%      1340.000000
75%      1340.000000
max      1340.000000
dtype: float64

In [11]:
base = base.fillna(method='ffill')
base = base.fillna(method='bfill')
base.head()

,005930.KS,0688.HK,0700.HK,1398.HK,2010.SR,7203.T,AAPL,AI.PA,AMT,AMZN,...,T,UNH,UNP,UPS,V,VOO,VTI,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,26600.0,24.704800,112.800003,5.77,79.50,7507.0,109.330002,89.786400,99.669998,308.519989,...,33.869999,100.779999,118.610001,110.379997,66.254997,188.399994,105.919998,46.959999,85.900002,92.830002
2015-01-05,26660.0,24.951799,113.500000,5.80,79.50,7507.0,106.250000,87.005997,98.230003,302.190002,...,33.549999,99.120003,114.599998,108.169998,64.792503,185.089996,104.099998,46.570000,85.650002,90.290001
2015-01-06,25900.0,24.605900,120.000000,5.71,77.00,7300.0,106.260002,86.279999,97.970001,295.290009,...,33.599998,98.919998,112.230003,107.459999,64.375000,183.270004,103.080002,47.040001,86.309998,89.809998
2015-01-07,26140.0,24.507099,124.400002,5.75,78.25,7407.0,107.750000,86.669601,99.000000,298.420013,...,33.169998,99.930000,112.849998,108.459999,65.237503,185.559998,104.309998,46.189999,88.599998,90.720001
2015-01-08,26280.0,23.864799,127.300003,5.72,79.25,7554.0,111.889999,90.317703,99.919998,300.459991,...,33.500000,104.699997,117.080002,110.410004,66.112503,188.820007,106.150002,47.180000,90.470001,92.230003


In [12]:
base.count().describe()

count      50.0
mean     1340.0
std         0.0
min      1340.0
25%      1340.0
50%      1340.0
75%      1340.0
max      1340.0
dtype: float64

Ya no hay NA's
La función queda como sigue:

In [136]:
def extraer_datos_yahoo(stocks):
    df_c = yf.download(stocks, start='2015-01-01', end='2020-04-30').Close
    base = df_c['AAPL'].dropna().to_frame()
    for i in range(0,50):
        base = base.join(df_c.iloc[:,i].to_frame(), lsuffix='_caller', rsuffix='_other')
    base = base.drop(columns=['AAPL_caller'])
    base = base.rename(columns={"AAPL_other": "AAPL"})
    base = base.fillna(method='ffill')
    base = base.fillna(method='bfill')
    return base

[*********************100%***********************]  6 of 6 completed


In [14]:
datos = extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 completed


In [15]:
datos.head()

,005930.KS,0688.HK,0700.HK,1398.HK,2010.SR,7203.T,AAPL,AI.PA,AMT,AMZN,...,T,UNH,UNP,UPS,V,VOO,VTI,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,26600.0,24.704800,112.800003,5.77,79.50,7507.0,109.330002,89.786400,99.669998,308.519989,...,33.869999,100.779999,118.610001,110.379997,66.254997,188.399994,105.919998,46.959999,85.900002,92.830002
2015-01-05,26660.0,24.951799,113.500000,5.80,79.50,7507.0,106.250000,87.005997,98.230003,302.190002,...,33.549999,99.120003,114.599998,108.169998,64.792503,185.089996,104.099998,46.570000,85.650002,90.290001
2015-01-06,25900.0,24.605900,120.000000,5.71,77.00,7300.0,106.260002,86.279999,97.970001,295.290009,...,33.599998,98.919998,112.230003,107.459999,64.375000,183.270004,103.080002,47.040001,86.309998,89.809998
2015-01-07,26140.0,24.507099,124.400002,5.75,78.25,7407.0,107.750000,86.669601,99.000000,298.420013,...,33.169998,99.930000,112.849998,108.459999,65.237503,185.559998,104.309998,46.189999,88.599998,90.720001
2015-01-08,26280.0,23.864799,127.300003,5.72,79.25,7554.0,111.889999,90.317703,99.919998,300.459991,...,33.500000,104.699997,117.080002,110.410004,66.112503,188.820007,106.150002,47.180000,90.470001,92.230003


**1.Sobre la documentación del código/de la función**

¿Se encuentran presentes en la implementación los siguientes elementos? Por favor, ingrese explicaciones detalladas.

**a) Descripción concisa y breve de lo que hace el código/la función**

La función no cuenta con descripción.

**b) Descripción de sus argumentos de entrada, su significado y rango de valores que pueden tomar**

Falta mencionar los argumentos, su significado y el rango de valores.

**c) Descripción de los tipos de argumentos de entrada y de salida (por ejemplo, valores enteros, reales, strings, dataframe, matrices, etc)**

[Pendiente: desarollo]

**d) Descripción de la salida de la función, su significado y valores/objetos que deben regresa**

[Pendiente: desarollo]


**2. Cumplimiento de objetivos del código/de la función**

Por favor, ingrese explicaciones detalladas.

**a) ¿El código cumple los objetivos para los que fue diseñado?**

Sí.

**b) ¿La salida de la función genera una DataFrame de tamaño #acciones $\times$ #días**

Sí.

**b) ¿La salida de la función genera un DataFrame cuyos elementos son accesibles en coordenadas (x,y)?**

Sí.

**3. Pruebas**

Ocupe la presente sección para hacer diseño de pruebas variando los parámetros que recibe el código la función en diferentes rangos para evaluae su comportamiento y/o detectar posibles fallos

**Test 1**

**Objetivo del test:** Revisar la descarga con distintas conjuntos de acciones y de fechas.

**Implementación del test:**

In [97]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

In [98]:
import random
random.seed(1234)

La función únicamente funciona para las 50 acciones y está definida para un intervalo específico de fechas. Modifcamos la función para seleccionar las acciones de cualquier intervalo de tiempo.

In [127]:
def extraer_datos_yahoo_test1(stocks, start='2015-01-01', end='2020-04-30'):
    df_c = yf.download(stocks,start,end ).Close
    base = df_c['AAPL'].dropna().to_frame()
    for i in range(0,50):
        base = base.join(df_c.iloc[:,i].to_frame(), lsuffix='_caller', rsuffix='_other')
    base = base.drop(columns=['AAPL_caller'])
    base = base.rename(columns={"AAPL_other": "AAPL"})
    base = base.fillna(method='ffill')
    base = base.fillna(method='bfill')
    return base

In [130]:
datos_1 = extraer_datos_yahoo_test1(stocks,start='2015-01-02', end='2020-05-05')
datos_1.head()

[*********************100%***********************]  50 of 50 completed


,005930.KS,0688.HK,0700.HK,1398.HK,2010.SR,7203.T,AAPL,AI.PA,AMT,AMZN,...,T,UNH,UNP,UPS,V,VOO,VTI,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,26600.0,24.704800,112.800003,5.77,79.50,7507.0,109.330002,89.786400,99.669998,308.519989,...,33.869999,100.779999,118.610001,110.379997,66.254997,188.399994,105.919998,46.959999,85.900002,92.830002
2015-01-05,26660.0,24.951799,113.500000,5.80,79.50,7507.0,106.250000,87.005997,98.230003,302.190002,...,33.549999,99.120003,114.599998,108.169998,64.792503,185.089996,104.099998,46.570000,85.650002,90.290001
2015-01-06,25900.0,24.605900,120.000000,5.71,77.00,7300.0,106.260002,86.279999,97.970001,295.290009,...,33.599998,98.919998,112.230003,107.459999,64.375000,183.270004,103.080002,47.040001,86.309998,89.809998
2015-01-07,26140.0,24.507099,124.400002,5.75,78.25,7407.0,107.750000,86.669601,99.000000,298.420013,...,33.169998,99.930000,112.849998,108.459999,65.237503,185.559998,104.309998,46.189999,88.599998,90.720001
2015-01-08,26280.0,23.864799,127.300003,5.72,79.25,7554.0,111.889999,90.317703,99.919998,300.459991,...,33.500000,104.699997,117.080002,110.410004,66.112503,188.820007,106.150002,47.180000,90.470001,92.230003


In [131]:
datos_2 = extraer_datos_yahoo_test1(stocks,start='2017-01-01', end='2019-01-01')
datos_2.head()

[*********************100%***********************]  50 of 50 completed


,005930.KS,0688.HK,0700.HK,1398.HK,2010.SR,7203.T,AAPL,AI.PA,AMT,AMZN,...,T,UNH,UNP,UPS,V,VOO,VTI,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,36480.0,20.750000,189.399994,4.68,92.50,7097.0,116.150002,96.045502,106.150002,753.669983,...,43.020000,161.449997,102.519997,115.080002,79.500000,206.740005,116.199997,54.580002,68.660004,90.889999
2017-01-04,36160.0,20.700001,189.000000,4.65,92.00,7097.0,116.019997,95.954498,106.339996,757.179993,...,42.770000,161.910004,103.139999,115.110001,80.150002,207.960007,117.089996,54.520000,69.059998,89.889999
2017-01-05,35560.0,21.000000,193.300003,4.71,92.50,7049.0,116.610001,96.000000,105.970001,780.450012,...,42.650002,162.179993,102.129997,115.169998,81.089996,207.800003,116.860001,54.639999,69.209999,88.550003
2017-01-06,36200.0,21.350000,195.100006,4.73,92.50,6930.0,117.910004,96.409103,105.269997,795.989990,...,41.320000,162.410004,103.190002,115.400002,82.209999,208.610001,117.230003,53.259998,68.260002,88.500000
2017-01-09,37220.0,21.650000,195.600006,4.68,92.25,6930.0,118.989998,95.181801,105.019997,796.919983,...,40.799999,161.949997,102.419998,114.769997,81.750000,207.949997,116.779999,52.680000,68.709999,87.040001


In [132]:
datos_3 = extraer_datos_yahoo_test1(stocks,start='2014-01-01', end='2019-01-01')
datos_3.head()

[*********************100%***********************]  50 of 50 completed


,005930.KS,0688.HK,0700.HK,1398.HK,2010.SR,7203.T,AAPL,AI.PA,AMT,AMZN,...,T,UNH,UNP,UPS,V,VOO,VTI,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,26180.0,21.641399,100.900002,5.20,113.25,6300.0,79.018570,81.261803,79.449997,397.970001,...,34.950001,74.570000,83.555000,103.320000,55.252499,167.630005,95.080002,49.000000,78.910004,99.750000
2014-01-03,25920.0,21.196699,98.800003,5.06,113.25,6300.0,77.282860,81.744797,79.680000,396.440002,...,34.799999,75.099998,83.425003,103.089996,55.290001,167.479996,95.059998,48.419998,78.650002,99.510002
2014-01-06,26140.0,20.653200,99.120003,4.96,113.00,6300.0,77.704285,80.094597,80.190002,393.630005,...,34.959999,74.239998,82.580002,101.750000,54.957500,167.059998,94.809998,48.689999,78.209999,99.660004
2014-01-07,26080.0,20.109699,98.800003,4.95,112.75,6270.0,77.148575,78.846901,81.300003,398.029999,...,34.950001,76.510002,83.309998,102.080002,55.377499,168.100006,95.419998,49.299999,78.449997,101.070000
2014-01-08,25820.0,21.443701,100.300003,5.04,115.50,6300.0,77.637146,79.506897,81.940002,401.920013,...,34.240002,75.620003,83.464996,102.000000,55.555000,168.169998,95.489998,48.500000,77.830002,100.739998


Principales hallazos del test:

* La función trabaja de manera para intervalos de tiempo distintos.

**Test 2***

**Objetivo del test:** Verificar si la función funciona para otras conjuntos de acciones.

**Implementación del test:**

In [134]:
a = np.random.randint(50,size= 1)
b = np.random.randint(50,size= 1)
min_p = min(a,b)
max_p = max(a,b)
stocks_1 = stocks[min_p[0]:max_p[0]]

In [135]:
datos_4 = extraer_datos_yahoo_test(stocks_1)

[*********************100%***********************]  28 of 28 completed


KeyError: "['AAPL_caller'] not found in axis"

In [137]:
datos_4 = extraer_datos_yahoo(stocks_1)

[*********************100%***********************]  28 of 28 completed


IndexError: single positional indexer is out-of-bounds

In [139]:
len(stocks_1)

28

In [140]:
def extraer_datos_yahoo_test_2(stocks, start='2015-01-01',end='2020-04-30'):
    #df_c = yf.download(stocks, start='2015-01-01', end='2020-04-30').Close
    df_c = yf.download(stocks, start, end).Close
    if 'AAPL' in stocks:
        base = df_c['AAPL'].dropna().to_frame()
        for i in range(0,len(stocks)):
            base = base.join(df_c.iloc[:,i].to_frame(), lsuffix='_caller', rsuffix='_other')
            base = base.drop(columns=['AAPL_caller'])
            base = base.rename(columns={"AAPL_other": "AAPL"})
            base = base.fillna(method='ffill')
            base = base.fillna(method='bfill')
        return base
    else:
        #base = df_c[''].to_frame()
        for i in range(0,len(stocks)):
            df_c = df_c.join(df_c.iloc[:,i].to_frame(), lsuffix='_caller', rsuffix='_other')
        return df_c

In [141]:
datos_5 = extraer_datos_yahoo_test_2(stocks_1)

[*********************100%***********************]  28 of 28 completed


KeyError: "['AAPL_caller'] not found in axis"

In [104]:
a = np.random.randint(50,size= 1)
b = np.random.randint(50,size= 1)
min_p = min(a,b)
max_p = max(a,b)
stocks_1 = stocks[min_p[0]:max_p[0]]

In [105]:
stocks_1

['COP',
 'AMT',
 'LIN',
 'LMT',
 'AMZN',
 'WMT',
 'JNJ',
 'VTI',
 'MSFT',
 'GOOG',
 'XOM',
 'CCI',
 'BHP.AX',
 'UNP',
 'BABA',
 'NSRGY',
 'RHHBY',
 'VOO',
 'AAPL']

In [112]:
datos_1 = extraer_datos_yahoo_test(stocks_1)

[*********************100%***********************]  19 of 19 completed


KeyError: "['AAPL_caller'] not found in axis"

SyntaxError: invalid syntax (<ipython-input-85-26b2a5324153>, line 13)

[*********************100%***********************]  6 of 6 completed


,0688.HK_caller,0700.HK_caller,BRK-A_caller,RDSA.AS_caller,UNH_caller,V_caller,0688.HK_other_caller,0700.HK_other_caller,BRK-A_other_caller,RDSA.AS_other_caller,...,UNH_other_other_other_other_other_other_other_caller,V_other_other_other_other_other_other_other_caller,0688.HK_other_other_other_other_other_other_other_other_caller,0700.HK_other_other_other_other_other_other_other_other_caller,BRK-A_other_other_other_other_other_other_other_other,RDSA.AS_other_other_other_other_other_other_other_other,UNH_other_other_other_other_other_other_other_other,V_other_other_other_other_other_other_other_other,0688.HK_other_other_other_other_other_other_other_other_other,0700.HK_other_other_other_other_other_other_other_other_other
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,24.704800,112.800003,223600.0,27.750000,100.779999,66.254997,24.704800,112.800003,223600.0,27.750000,...,100.779999,66.254997,24.704800,112.800003,223600.0,27.750000,100.779999,66.254997,24.704800,112.800003
2015-01-05,24.951799,113.500000,220980.0,26.615000,99.120003,64.792503,24.951799,113.500000,220980.0,26.615000,...,99.120003,64.792503,24.951799,113.500000,220980.0,26.615000,99.120003,64.792503,24.951799,113.500000
2015-01-06,24.605900,120.000000,220450.0,26.514999,98.919998,64.375000,24.605900,120.000000,220450.0,26.514999,...,98.919998,64.375000,24.605900,120.000000,220450.0,26.514999,98.919998,64.375000,24.605900,120.000000
2015-01-07,24.507099,124.400002,223480.0,26.870001,99.930000,65.237503,24.507099,124.400002,223480.0,26.870001,...,99.930000,65.237503,24.507099,124.400002,223480.0,26.870001,99.930000,65.237503,24.507099,124.400002
2015-01-08,23.864799,127.300003,226680.0,27.495001,104.699997,66.112503,23.864799,127.300003,226680.0,27.495001,...,104.699997,66.112503,23.864799,127.300003,226680.0,27.495001,104.699997,66.112503,23.864799,127.300003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-23,26.200001,411.600006,278750.0,16.521999,285.329987,166.380005,26.200001,411.600006,278750.0,16.521999,...,285.329987,166.380005,26.200001,411.600006,278750.0,16.521999,285.329987,166.380005,26.200001,411.600006
2020-04-24,26.100000,406.399994,279460.0,15.978000,291.290009,167.320007,26.100000,406.399994,279460.0,15.978000,...,291.290009,167.320007,26.100000,406.399994,279460.0,15.978000,291.290009,167.320007,26.100000,406.399994
2020-04-27,26.950001,413.000000,281264.0,16.250000,293.980011,171.759995,26.950001,413.000000,281264.0,16.250000,...,293.980011,171.759995,26.950001,413.000000,281264.0,16.250000,293.980011,171.759995,26.950001,413.000000


**4. Resumen detallado de posibles puntos faltantes en implementación**

* La función no tiene documentación que explique los argumentos de extrada, salida ni el rango de valores que puede tomar.
* Modifi

**Sugerencias para resolver los puntos anteriores**

* [Pendiente: desarollo]
* [Pendiente: desarollo]
* [Pendiente: desarollo]